In [ ]:
from tcd_pipeline.modelrunner import ModelRunner

Begin by instantiating a `ModelRunner` object which handles model loading from a configuration file. You can see examples of these files in the `config` folder. You use the same object if you want to launch a semantic segmentation or an instance segmentation model, but a different configuration file.

In [ ]:
runner = ModelRunner("../config/base_semantic_segmentation.yaml")

You call the `predict` method on an image, which can be provided as a path to a GeoTIFF or a Rasterio dataset. Since our pipeline requires some knowledge of spatial scale, the data should be georeferenced. If you just want to test an image, you can "fake" the georeferencing data.

In [ ]:
image_path = "../data/5c15321f63d9810007f8b06f_10_00000.tif"
res = runner.predict(image_path)

The `res` object can be used to serialise and visualise data, as well as generate summary statistics.

In [ ]:
res.visualise()

In [ ]:
res.canopy_cover

If you want to access raw data products, you can look at the canopy mask or confidence maps:

In [ ]:
res.canopy_mask

In [ ]:
res.confidence_map

You can save data to a file using `serialise`

In [ ]:
res.serialise("/tmp/prediction_test")

In [ ]:
!ls /tmp/prediction_test

In [ ]:
!cat /tmp/prediction_test/results.json